In [20]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input, ctx
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import time
import ta

import oandapyV20
from oandapyV20 import API
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.trades as trades
import oandapyV20.endpoints.accounts as accounts


In [2]:
account_id = "ACCOUNT_ID"
token = "TOKEN"
api = oandapyV20.API(access_token=token)

In [122]:
def get_currency_price(instrument_list):
    rst_dict = {}
    for instrument in instrument_list:
        params = {"instruments": instrument, "since": None, "includeUnitsAvailable": False}
        r = pricing.PricingInfo(accountID=account_id, params=params)
        api.request(r)

        price = r.response.get("prices")[0]
        
        rst_dict[instrument] = {}
        rst_dict[instrument]["bid"] = float(price.get("bids")[0].get("price"))
        rst_dict[instrument]["ask"] = float(price.get("asks")[0].get("price"))

    return rst_dict

In [126]:
default_count = 50
def get_currency_candle(instrument, granularity = 'M1', count = default_count):
    df = pd.DataFrame()
    params = {"granularity": granularity, "count": count}
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    api.request(r)

    data = [
        {
            "Time": pd.to_datetime(d["time"].split(".")[0]),
            "High": float(d["mid"]["h"]),
            "Close": float(d["mid"]["c"]),
            "Low": float(d["mid"]["l"]),
            "Open": float(d["mid"]["o"]),
        }
        for d in r.response["candles"]
    ]
    df = pd.DataFrame(data)
    return df

In [7]:
def get_open_trade():
    df = pd.DataFrame()
    r = trades.OpenTrades(accountID=account_id)
    api.request(r)
    data = [
        {
            "Ticket": d["id"],
            "Market": d["instrument"].split("_")[0] + "/" + d["instrument"].split("_")[1],
            "Units": abs(int(d["currentUnits"])),
            # if the currentUnits is negative, it is short position
            "Type": "Short" if int(d["currentUnits"]) < 0 else "Long",
            "Price": d["price"],
            "Profit": round(float(d["unrealizedPL"]), 2),
        }
        for d in r.response["trades"]
    ]
    df = pd.DataFrame(data)
    return df

In [9]:
def close_all_trade(trade_id_list):
    msg = []
    for trade_id in trade_id_list:
        r = trades.TradeClose(accountID=account_id, tradeID=trade_id)
        api.request(r)
        msg.append(r.response)
        time.sleep(0.3)
    return msg

In [11]:
def get_account_info():
    r = accounts.AccountDetails(accountID=account_id)
    api.request(r)
    return r.response

In [114]:
def format_number(num, unit="SGD"):
    return str(num) + " " + unit

In [127]:
currency_pair_list = ['EUR_USD', 'GBP_USD', 'USD_JPY', 'AUD_USD']
granularity_value_list = ['S10', 'M1', 'M5', 'M30', 'H4', 'H8', 'D', 'W', 'M']
granularity_label_list = ['10 secs', '1 min', '5 mins', '30 mins', '4 hours', '8 hours', 'Day', 'Week', 'Month']

In [129]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.H1('Forex Trading Dashboard', style={'text-align': 'center', 'padding-top': '15px', 'padding-bottom': '5px'}),
    # Add a horizontal line
    html.Hr(),
    dbc.Row(
        [
            # Account Information: NAV, Balance, Unrealized P/L, Realized P/L
            dbc.Col(dbc.Card([dbc.CardHeader(html.H3("NAV", className="card-text")),
                                dbc.CardBody(html.H4(id='NAV', className="card-text")),], color="dark", outline=True), width="auto", 
                     style={"width": "18rem"},),
            dbc.Col(dbc.Card([dbc.CardHeader(html.H3("Balance", className="card-text")),
                                dbc.CardBody(html.H4(id='balance', className="card-text")),], color="dark", outline=True), width="auto", 
                     style={"width": "18rem"},),
            dbc.Col(dbc.Card([dbc.CardHeader(html.H3("Unrealized P/L", className="card-text")),
                                dbc.CardBody(html.H4(id='unrealizedPL', className="card-text")),], color="dark", outline=True), width="auto", 
                     style={"width": "18rem"},),
            dbc.Col(dbc.Card([dbc.CardHeader(html.H3("Realized P/L", className="card-text")),
                                dbc.CardBody(html.H4(id='realizedPL', className="card-text")),], color="dark", outline=True), width="auto", 
                     style={"width": "18rem"},),
        ],
        # set all the component to center
        style={"display": "flex", "justify-content": "center", "align-items": "center"}
    ),
    html.Br(),
    
    dbc.Row(
        [
            dbc.Col(html.Div(id='close-all-trade-msg'), width="auto"),
            # show radio button to select the currency pair
            dbc.Col(dbc.RadioItems(id='currency-pair', options=[{'label': i.replace('_', '/'), 'value': i} for i in currency_pair_list], value='EUR_USD', inline=True), width="auto"),
            # add a dropdown to select the granularity of the candlestick chart
            dbc.Col(dbc.Select(id='granularity', options=[{'label': label, 'value': value} for label, value in zip(granularity_label_list, granularity_value_list)], value='M1'), width="auto"),
            # add a dropdown to select which indicator to calculate
            dbc.Col(dbc.Select(id='indicator', options=[{'label': i, 'value': i} for i in ['RSI', 'MACD', 'SMA', 'Stochastic Oscillator']], value='RSI'), width="auto"),
        ],
        style={"margin-left": "90px"}
    ),

    html.Br(),
    
    dbc.Row(
        [
            dbc.Col([
                # show the candlestick chart of currency price
                dcc.Graph(id='candlestick-chart'),
                # To Do: add a dropdown to select window size of the indicator
                # show the indicator chart
                dcc.Graph(id='indicator-chart'),],
                style={"margin-left": "0px", "margin-right": "0px"}
            ),
            dbc.Col([
                # show the latest currency pair bid/ask price and spread
                html.H3('Currency Price'),
                dbc.Table([
                    html.Tr([html.Th('Currency Pair'), html.Th('Bid Price'), html.Th('Ask Price'), html.Th('Spread')]),
                    html.Tr([html.Td('EUR/USD'), html.Td(id='EURUSD-bid-price'), html.Td(id='EURUSD-ask-price'), html.Td(id="EURUSD-spread")]),
                    html.Tr([html.Td('GBP/USD'), html.Td(id='GBPUSD-bid-price'), html.Td(id='GBPUSD-ask-price'), html.Td(id="GBPUSD-spread")]),
                    html.Tr([html.Td('USD/JPY'), html.Td(id='USDJPY-bid-price'), html.Td(id='USDJPY-ask-price'), html.Td(id="USDJPY-spread")]),
                    html.Tr([html.Td('AUD/USD'), html.Td(id='AUDUSD-bid-price'), html.Td(id='AUDUSD-ask-price'), html.Td(id="AUDUSD-spread")]),
                ], color="primary", bordered=True, striped=True, size="lg"),
                # show open trade information in datatable
                html.H3('Open Position Information'),
                dash_table.DataTable(id='open-trade-table', style_cell={'textAlign': 'left', 'font_family': 'sans-serif'}, style_header={'fontWeight': 'bold'}),
                html.Br(),
                dbc.Button('Close All Trade', id='close-all-trade-button', n_clicks=0),
            ], 
            style={"margin-left": "0px", "margin-right": "0px"}),
        ],
        style={"margin-left": "20px", "margin-right": "20px"}
    ),
    # update the chart every second
    dcc.Interval(id='interval', interval=5000)
])

#================================================================
# Update the candlestick chart with the selected currency pair
#================================================================
@app.callback(
    Output(component_id='candlestick-chart', component_property='figure'),
    Input(component_id='currency-pair', component_property='value'),
    Input(component_id='granularity', component_property='value'),
    Input(component_id='interval', component_property='n_intervals')
)
def update_candlestick_chart(currency_pair, granularity, n_intervals):
    # get the currency pair selected
    df = get_currency_candle(currency_pair, granularity=granularity)
    fig = go.Figure(data=[go.Candlestick(x=df['Time'],
                                         open=df['Open'],
                                         high=df['High'],
                                         low=df['Low'],
                                         close=df['Close'])])
    # fig.update_layout(title='Candlestick Chart', xaxis_title='Time', yaxis_title='Price', xaxis_rangeslider_visible=False, height=400, width=1300)
    fig.update_layout(yaxis_title='Price', xaxis_rangeslider_visible=False, height=400, width=1300, margin=dict(t=0, b=0))

    return fig

#================================================================
# Update the currency pair price
#================================================================
@app.callback(
    Output(component_id='EURUSD-bid-price', component_property='children'),
    Output(component_id='EURUSD-ask-price', component_property='children'),
    Output(component_id='GBPUSD-bid-price', component_property='children'),
    Output(component_id='GBPUSD-ask-price', component_property='children'),
    Output(component_id='USDJPY-bid-price', component_property='children'),
    Output(component_id='USDJPY-ask-price', component_property='children'),
    Output(component_id='AUDUSD-bid-price', component_property='children'),
    Output(component_id='AUDUSD-ask-price', component_property='children'),
    Output(component_id='EURUSD-spread', component_property='children'),
    Output(component_id='GBPUSD-spread', component_property='children'),
    Output(component_id='USDJPY-spread', component_property='children'),
    Output(component_id='AUDUSD-spread', component_property='children'),
    Input(component_id='interval', component_property='n_intervals')
)
def update_currency_price(n_intervals):
    # get the latest currency price
    currency_price_dict = get_currency_price(currency_pair_list)

    return currency_price_dict.get('EUR_USD').get('bid'), currency_price_dict.get('EUR_USD').get('ask'), \
        currency_price_dict.get('GBP_USD').get('bid'), currency_price_dict.get('GBP_USD').get('ask'), \
        currency_price_dict.get('USD_JPY').get('bid'), currency_price_dict.get('USD_JPY').get('ask'), \
        currency_price_dict.get('AUD_USD').get('bid'), currency_price_dict.get('AUD_USD').get('ask'), \
        round(currency_price_dict.get('EUR_USD').get('ask') - currency_price_dict.get('EUR_USD').get('bid'), 5), \
        round(currency_price_dict.get('GBP_USD').get('ask') - currency_price_dict.get('GBP_USD').get('bid'), 5), \
        round(currency_price_dict.get('USD_JPY').get('ask') - currency_price_dict.get('USD_JPY').get('bid'), 3), \
        round(currency_price_dict.get('AUD_USD').get('ask') - currency_price_dict.get('AUD_USD').get('bid'), 5)


#================================================================
# Update the open trade information
#================================================================
@app.callback(
    Output(component_id='open-trade-table', component_property='data'),
    Input(component_id='interval', component_property='n_intervals')
)
def list_open_trade(n_intervals):
    # get the open trade information
    open_trade_df = get_open_trade()

    return open_trade_df.to_dict('records')

#================================================================
# Close all trade if the button is clicked
#================================================================
@app.callback(
    Output(component_id='close-all-trade-msg', component_property='children'),
    Input(component_id='close-all-trade-button', component_property='n_clicks')
)
def close_all_trade(all_trade_button_click):
    # get the open trade information
    open_trade_df = get_open_trade()
    # close all trade if the button is clicked
    msg = ""
    if "close-all-trade-button" == ctx.triggered_id:
        msg = close_all_trade(open_trade_df['Ticket'].tolist())

    return msg

#================================================================
# Update the indicator chart with the selected indicator
#================================================================
@app.callback(
    Output(component_id='indicator-chart', component_property='figure'),
    Input(component_id='indicator', component_property='value'),
    Input(component_id='interval', component_property='n_intervals'),
    Input(component_id='currency-pair', component_property='value'),
    Input(component_id='granularity', component_property='value')
)
def update_indicator_chart(indicator, n_intervals, currency_pair, granularity):
    max_window_size = 20
    # get the currency pair open, high, low, close price
    df = get_currency_candle(currency_pair, granularity=granularity, count=default_count+max_window_size)
    # calculate the indicator
    if indicator == 'RSI':
        df['RSI'] = ta.momentum.rsi(df['Close'], window=5)
        fig = px.line(df[max_window_size:default_count+max_window_size], x='Time', y='RSI')
    elif indicator == 'MACD':
        df['MACD'] = ta.trend.macd_diff(df['Close'], window_slow=13, window_fast=5)
        fig = px.line(df[max_window_size:default_count+max_window_size], x='Time', y='MACD')
    elif indicator == 'SMA':
        df['SMA'] = ta.trend.sma_indicator(df['Close'], window=5)
        fig = px.line(df[max_window_size:default_count+max_window_size], x='Time', y='SMA')
    elif indicator == 'Stochastic Oscillator':
        df['Stochastic Oscillator'] = ta.momentum.stoch(df['High'], df['Low'], df['Close'], window=5)
        fig = px.line(df[max_window_size:default_count+max_window_size], x='Time', y='Stochastic Oscillator')
    
    fig.update_layout(height=300, width=1300, margin=dict(t=0))
    return fig

#================================================================
# Update the account information (NAV, balance, unrealizedPL, realizedPL)
#================================================================
@app.callback(
    Output(component_id='NAV', component_property='children'),
    Output(component_id='balance', component_property='children'),
    Output(component_id='unrealizedPL', component_property='children'),
    Output(component_id='realizedPL', component_property='children'),
    Input(component_id='interval', component_property='n_intervals')
)
def update_account_info(n_intervals):
    # get the account information
    acc_info = get_account_info()
    balance = round(float(acc_info['account']['balance']), 2)
    realizedPL = round(float(acc_info['account']['pl']), 2)
    # get the open trade information
    open_trade_df = get_open_trade()
    # if there are no data in open_trade_df, unrealizedPL is 0
    if open_trade_df.empty:
        unrealizedPL = 0
    else:
        unrealizedPL = round(open_trade_df['Profit'].sum(), 2)
    nav = round(balance + unrealizedPL, 2)

    return format_number(nav), format_number(balance), format_number(unrealizedPL), format_number(realizedPL)

if __name__ == '__main__':
    app.run_server(jupyter_mode="external", debug=True)

Dash app running on http://127.0.0.1:8050/
